<h1> Booking.com </h1>
<h3> <i>Hotels in Goa </i></h3>
<hr/><br/>

<b>Fields to extract: </b>
<ol> 
    <li> Hotel names </li>
    <li> Room type </li>
    <li> Room price </li>
    <li> Location </li>
    <li> User reviews </li>
    <li> User ratings </li>
    <li> Photos </li>
    <li> Description </li>
</ol>
<hr/>



In [16]:
# import necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [17]:
# target url to scrap
# Booking.com + hotels_in_goa
url = 'https://www.booking.com/searchresults.en-gb.html?aid=304142&label=gen173rf-1FCAEoggI46AdIM1gDaAKIAQGYAQm4AQjIAQzYAQHoAQH4AQuIAgGiAg9pbnRlcm5zaGFsYS5jb22oAgO4ApGSkPwFwAIB0gIkODllNzg5YTMtNGYxZi00MDU3LTg0NzItMzBmNzVhYTEyZDUz2AIG4AIB&sid=c58bbadfd6e1c32ace53a3c4b79e65fe&tmpl=searchresults&class_interval=1&dest_id=4127&dest_type=region&from_sf=1&group_adults=1&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=region&room1=A&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=f5ea40d9a19b0020&ss=Goa&ssb=empty&ssne=Goa&ssne_untouched=Goa&top_ufis=1&rows=25&offset=' 


In [18]:
# send request to download the data
response = requests.request('GET', url+'0')

# parse the data using BeautifulSoup
data = BeautifulSoup(response.text, 'html.parser')


In [19]:
data

<!DOCTYPE html>

<!--
You know you could be getting paid to poke around in our code?
We're hiring designers and developers to work in Amsterdam:
https://careers.booking.com/
-->
<!-- wdot-802 -->
<script nonce="vX6Dc1HbuHN2IAs" type="text/javascript">
document.addEventListener('DOMContentLoaded', function () {
//c360 javascript tracker first iteration
//sends a track request to c360 http tracker
//in order to use it:
//1. inline the c360Tracker.js in the page you need to use it
//2. in your js file:
//
// var c360Tracker = B.require('c360Tracker');
// var event = {
// action_name:"accommodation_checkout_confirmation_viewed",
// action_version :"0.2.0",
// content : { "transaction_id" : 123434},
// user : { "BKNG_user_id": 123434}
// };
// c360Tracker.track(event);
B.define('c360Tracker', function () {
var enrichedContext = {};
var configuration = {
validateInput: false
};
var track = function (event) {
if (event == null) {
return "event object is null or empty";
} else {
//if (enriched

<b>h1</b> tag corresponds to the main heading in the web page. Here, we have the no of hotels in the h1 tag. So we can extract the no of hotels from the text within the h1 tag as follows:

<h1 class="e1f827110f d3a14d00da">Goa: 2,379 properties found</h1>
    

In [20]:
data.find('h1')

<h1 class="e1f827110f d3a14d00da">Goa: 2,260 properties found</h1>

In [21]:
data.find('h1').text.split(':')

['Goa', ' 2,260 properties found']

In [22]:
data.find('h1').text.split(':')[1]

' 2,260 properties found'

In [23]:
data.find('h1').text.split(':')[1].split()

['2,260', 'properties', 'found']

In [24]:
data.find('h1').text.split(':')[1].split()[0]

'2,260'

In [25]:
data.find('h1').text.split(':')[1].split()[0].replace(',', '').strip()

'2260'

In [26]:
int(data.find('h1').text.split(':')[1].split()[0].replace(',', '').strip())

2260

In [27]:
#find the total no of hotels
no_hotels = int(data.find('h1').text.split(':')[1].split()[0].replace(',', '').strip())

#clear the contents in the file
hotels_file = open('Hotels_in_Goa.csv', 'w')
rooms_file = open('Room_types.csv', 'w')
hotels_file.truncate()
rooms_file.truncate()
hotels_file.close()
rooms_file.close()

print(no_hotels)

2260


The following is a small code snippet, which will help you understand how the information corresponding to hotels in a single page are extracted one at a time.

In [28]:
print('---------------------------------------- Extracted: {:.2f}% ----------------------------------------'.format(0))

# hotels_headers = ['name', 'location', 'user_reviews_score', 'user_reviews_title', 'user_reviews_no', 'is_preferred_partner_property', 'user_rating_in_stars', 'description', 'image_src', 'hotel_url']
# room_types_headers = ['hotel_name', 'room_type', 'bed_type']


# Each page has info regarding 25 hotels
# At the end of the url, u can find 'OFFSET='
# The OFFSET=0 for page1, 25 for page 2, 50 for page 3, etc...
# So the offset increments by 25, while the page nos increment by 1

for offset in range(0, 1):  
    
    # send request to download the data
    hotels_response = requests.request('GET', url + str(offset))
    
    # parse the data using BeautifulSoup
    hotels_data = BeautifulSoup(hotels_response.text, 'html.parser')

    # find the individual hotel cards
    cards = hotels_data.find_all('div', attrs={'data-testid': 'property-card'})
        
    # we will create a list of dictionaries, each list item or dictionary will correspond to a hotel
    hotels_details = []
    room_types = []
    
    
    for card in cards:
        # create a dictionary for each hotel entry
        hotel_details_dict = dict()
        
        # get the field values from the website and store in the dictionary
        name = None if card.find('div', attrs={'data-testid': 'title'}) is None \
                    else card.find('div', attrs={'data-testid': 'title'}).text.strip()
        hotel_details_dict['name'] = name
        hotel_details_dict['location'] = None if card.find('span', attrs={'data-testid': 'address'}) is None \
                                              else card.find('span', attrs={'data-testid': 'address'}).contents[0].strip()
        card_hotel_reviews = card.find('div', attrs={'data-testid': 'review-score'})
        hotel_details_dict['user_reviews_score'] = None if card_hotel_reviews is None \
                                                        else card_hotel_reviews.find('div').text.strip()
        hotel_details_dict['user_reviews_title'] = None if card.find('div', attrs={'data-testid': 'review-score'}) is None \
                                                        else card_hotel_reviews.find_all('div')[1].find('div').text.strip()
        hotel_details_dict['user_reviews_no'] = None if card.find('div', attrs={'data-testid': 'review-score'}) is None \
                                                        else card_hotel_reviews.find_all('div')[1].find_all('div')[1].text.split()[0].strip()
        hotel_details_dict['is_preferred_partner_property'] = False if card.find('span', attrs={'data-testid': 'preferred-badge'}) is None \
                                                                    else True
        if (card.find('div', attrs={'data-testid': 'rating-squares'}) is None):
            if (card.find('div', attrs={'data-testid': 'rating-stars'}) is None):
                hotel_details_dict['user_rating_in_stars'] = None
            else:
                hotel_details_dict['user_rating_in_stars'] = len(card.find('div', attrs={'data-testid': 'rating-stars'}).find_all('span'))                
        else:
            hotel_details_dict['user_rating_in_stars'] = len(card.find('div', attrs={'data-testid': 'rating-squares'}).find_all('span'))
        hotel_details_dict['description'] = None if card.find('div', attrs={'class': 'd8eab2cf7f'}) is None \
                                                 else card.find('div', attrs={'class': 'd8eab2cf7f'}).text.strip()
        hotel_details_dict['image_src'] = None if card.find('img', attrs={'data-testid':'image'}) is None \
                                            else card.find('img', attrs={'data-testid':'image'}).get('src').strip()
        hotel_details_dict['hotel_url'] = card.find('a', attrs={'data-testid': 'title-link'}).get('href').strip()
        
        for key,value in hotel_details_dict.items():
            print(key, ': ', value)
        print('\n----------------------\n\n')

    
    

---------------------------------------- Extracted: 0.00% ----------------------------------------
name :  JM Vista Suites
location :  Siolim
user_reviews_score :  9.0
user_reviews_title :  Superb
user_reviews_no :  19
is_preferred_partner_property :  True
user_rating_in_stars :  None
description :  Travel Sustainable property
image_src :  https://cf.bstatic.com/xdata/images/hotel/square200/180182886.jpg?k=bf043c6bb764d71ab494f717e27da7e8f08e34afb030912d0b5e946712861de4&o=
hotel_url :  https://www.booking.com/hotel/in/jm-vista-suites.en-gb.html?ucfs=1&arphpl=1&dest_id=4127&dest_type=region&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=6632467ad8aa048f&srepoch=1654682485&from_sustainable_property_sr=1&from=searchresults#hotelTmpl

----------------------


name :  III Kings Morjim
location :  Pernem
user_reviews_score :  None
user_reviews_title :  None
user_reviews_no :  None
is_preferred_partner_property :  True
user_rat

The following is the original code to extract all the features pertaing to all the hotels in the website meeting the criteris as in the problem statement.

In [29]:
print('---------------------------------------- Extracted: {:.2f}% ----------------------------------------'.format(0))

# hotels_headers = ['name', 'location', 'user_reviews_score', 'user_reviews_title', 'user_reviews_no', 'is_preferred_partner_property', 'user_rating_in_stars', 'description', 'image_src', 'hotel_url']
# room_types_headers = ['hotel_name', 'room_type', 'bed_type']


# Each page has info regarding 25 hotels
# At the end of the url, u can find 'OFFSET='
# The OFFSET=0 for page1, 25 for page 2, 50 for page 3, etc...
# So the offset increments by 25, while the page nos increment by 1

for offset in range(0, no_hotels, 25):
    
    # send request to download the data
    hotels_response = requests.request('GET', url + str(offset))
    
    # parse the data using BeautifulSoup
    hotels_data = BeautifulSoup(hotels_response.text, 'html.parser')

    # find the individual hotel cards
    cards = hotels_data.find_all('div', attrs={'data-testid': 'property-card'})
        
    # we will create a list of dictionaries, each list item or dictionary will correspond to a hotel
    hotels_details = []
    room_types = []
    
    
    for card in cards:
        # create a dictionary for each hotel entry
        hotel_details_dict = dict()
        
        # get the field values from the website and store in the dictionary
        name = None if card.find('div', attrs={'data-testid': 'title'}) is None \
                    else card.find('div', attrs={'data-testid': 'title'}).text.strip()
        hotel_details_dict['name'] = name
        hotel_details_dict['location'] = None if card.find('span', attrs={'data-testid': 'address'}) is None \
                                              else card.find('span', attrs={'data-testid': 'address'}).contents[0].strip()
        card_hotel_reviews = card.find('div', attrs={'data-testid': 'review-score'})
        hotel_details_dict['user_reviews_score'] = None if card_hotel_reviews is None \
                                                        else card_hotel_reviews.find('div').text.strip()
        hotel_details_dict['user_reviews_title'] = None if card.find('div', attrs={'data-testid': 'review-score'}) is None \
                                                        else card_hotel_reviews.find_all('div')[1].find('div').text.strip()
        hotel_details_dict['user_reviews_no'] = None if card.find('div', attrs={'data-testid': 'review-score'}) is None \
                                                        else card_hotel_reviews.find_all('div')[1].find_all('div')[1].text.split()[0].strip()
        hotel_details_dict['is_preferred_partner_property'] = False if card.find('span', attrs={'data-testid': 'preferred-badge'}) is None \
                                                                    else True
        if (card.find('div', attrs={'data-testid': 'rating-squares'}) is None):
            if (card.find('div', attrs={'data-testid': 'rating-stars'}) is None):
                hotel_details_dict['user_rating_in_stars'] = None
            else:
                hotel_details_dict['user_rating_in_stars'] = len(card.find('div', attrs={'data-testid': 'rating-stars'}).find_all('span'))                
        else:
            hotel_details_dict['user_rating_in_stars'] = len(card.find('div', attrs={'data-testid': 'rating-squares'}).find_all('span'))
        hotel_details_dict['description'] = None if card.find('div', attrs={'class': 'd8eab2cf7f'}) is None \
                                                 else card.find('div', attrs={'class': 'd8eab2cf7f'}).text.strip()
        hotel_details_dict['image_src'] = None if card.find('img', attrs={'data-testid':'image'}) is None \
                                            else card.find('img', attrs={'data-testid':'image'}).get('src').strip()
        hotel_url = card.find('a', attrs={'data-testid': 'title-link'}).get('href').strip()
        hotel_details_dict['hotel_url'] = hotel_url
                
        # send request to get other deatils related to the hotel
        hotel_response = requests.request('GET', hotel_url)
        hotel_data = BeautifulSoup(hotel_response.text, 'html.parser')

        room_details_table = None if hotel_data.find('table', attrs={'id': 'maxotel_rooms'}) is None \
                                  else hotel_data.find('table', attrs={'id': 'maxotel_rooms'}).find('tbody').find_all('tr')[::2]
        
        for room_details in room_details_table:
            room_types_dict = dict()
            room_types_dict['hotel_name'] = name  
            room_types_dict['room_type'] = None if room_details.find('a', attrs={'class': 'jqrt togglelink'}) is None \
                                                else room_details.find('a', attrs={'class': 'jqrt togglelink'}).get('data-room-name-en')
            if room_details.find('div', attrs={'class': 'bed-types-wrapper'}) is None:
                if room_details.find('div', attrs={'class': 'appartment-bed-types-wrapper'}) is None:
                    room_types_dict['bed_type'] = None
                else:
                    room_types_dict['bed_type'] = room_details.find('div', attrs={'class': 'appartment-bed-types-wrapper'}).text.strip().replace('\n', ' ')
            else:
                room_types_dict['bed_type'] = room_details.find('div', attrs={'class': 'bed-types-wrapper'}).text.strip().replace('\n', ' ')
                    
            occupancy = None if room_details.find('span', attrs={'class': 'bui-u-sr-only'}) is None \
                             else room_details.find('span', attrs={'class': 'bui-u-sr-only'}).text.split('.')
            if occupancy is None:
                max_occupancy_adults = 0
                max_occupancy_children = 0
            elif len(occupancy) == 1:
                max_occupancy_adults = int(occupancy[0].split(':')[1].strip())
                max_occupancy_children = 0
            else:
                max_occupancy_adults = int(occupancy[0].split(':')[1].strip())
                max_occupancy_children = int(occupancy[1].split(':')[1].strip())
            room_types_dict['max_occupancy_adults'] = max_occupancy_adults
            room_types_dict['max_occupancy_children'] = max_occupancy_children

            room_types.append(room_types_dict)
            
        # append the dictionay (hotel details) to the list
        hotels_details.append(hotel_details_dict)
            
    # Convert the list of dictionaries to dataframes 
    hotels = pd.DataFrame.from_dict(hotels_details)
    rooms = pd.DataFrame.from_dict(room_types)
    rooms.set_index(['hotel_name', 'room_type'], inplace=True)
    
    # Save the dataframes in csv format
    hotels.to_csv('Hotels_in_Goa.csv', mode='a', header=True if offset==0 else False, index=False)
    rooms.to_csv('Room_types.csv', mode='a', header=True if offset==0 else False)

    percent_done = ((offset+len(cards))/(no_hotels-1)) * 100
  
    print('---------------------------------------- Extracted: {:.2f}% ----------------------------------------'.format(percent_done,offset, no_hotels))
        
    

---------------------------------------- Extracted: 0.00% ----------------------------------------
---------------------------------------- Extracted: 1.11% ----------------------------------------
---------------------------------------- Extracted: 2.21% ----------------------------------------
---------------------------------------- Extracted: 3.32% ----------------------------------------
---------------------------------------- Extracted: 4.43% ----------------------------------------
---------------------------------------- Extracted: 5.53% ----------------------------------------


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))